In [1]:
import io
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser
import mysql.connector as mysqlC
import pymysql

In [2]:
config = configparser.ConfigParser()
config.read('config/escec.cfg')

['config/escec.cfg']

## Config RDS

In [3]:
RDS_HOST = 'db-rent-cars.chf84lio5m7c.us-east-1.rds.amazonaws.com'
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""  

## Config S3

In [5]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

S3_BUCKET_NAME = 'bucket-23000712'

## Date dim

In [6]:
try:
    file = s3.Bucket(S3_BUCKET_NAME).Object('dim_date.xlsx').get()
    data = file['Body'].read()
    df_dim_date = pd.read_excel(io.BytesIO(data), engine='openpyxl')
except Exception as ex:
    print("No es un archivo.")
    print(ex)

# obtiene solo las columnas listadas
df_dim_date = df_dim_date[['date key','full date','day of week','day num in month',	'day name','quarter','year']]

# renombra las columnas
df_dim_date = df_dim_date.rename(columns={'date key':'date_key','full date':'full_date','day of week':'day_of_week','day num in month':'day_num_in_month',	'day name':'day_name','quarter':'quarter','year':'year'})

df_dim_date.head()

,date_key,full_date,day_of_week,day_num_in_month,day_name,quarter,year
0,20150101,2015-01-01,4,1,Thursday,1,2015
1,20150102,2015-01-02,5,2,Friday,1,2015
2,20150103,2015-01-03,6,3,Saturday,1,2015
3,20150104,2015-01-04,7,4,Sunday,1,2015
4,20150105,2015-01-05,1,5,Monday,1,2015


## Location dim

In [7]:
try:
    file = s3.Bucket(S3_BUCKET_NAME).Object('dim_state.xlsx').get()
    data = file['Body'].read()
    df_state = pd.read_excel(io.BytesIO(data), engine='openpyxl')
except Exception as ex:
    print("No es un archivo.")
    print(ex)

df_state.head()

,id,state
0,AL,Alabama
1,AK,Alaska
2,AS,American Samoa
3,AZ,Arizona
4,AR,Arkansas


In [8]:
sql_query = 'SELECT * FROM location;'
df_location = pd.read_sql(sql_query, mysql_driver)
df_location.head()

,id,street_address,city,state,zipcode
0,1,1001 Henderson St,Fort Worth,TX,76102
1,2,300 Reunion Blvd,Dallas,TX,75207
2,3,5911 Blair Rd NW,Washington,DC,20011
3,4,9217 Airport Blvd,Los Angeles,CA,90045
4,5,310 E 64th St,New York,NY,10021


In [9]:
df_dim_location = df_location.merge(df_state, 
                                                left_on='state',
                                                right_on='id',
                                                how='left',
                                                suffixes=('_loc','_sta'))
df_dim_location.drop(['state_loc', 'zipcode', 'id_sta'], axis=1, inplace=True)
df_dim_location.rename(columns={'state_sta': 'state'}, inplace=True)
df_dim_location.head()

,id_loc,street_address,city,state
0,1,1001 Henderson St,Fort Worth,Texas
1,2,300 Reunion Blvd,Dallas,Texas
2,3,5911 Blair Rd NW,Washington,Disctrict of Columbia
3,4,9217 Airport Blvd,Los Angeles,California
4,5,310 E 64th St,New York,New York


## Customer dim

In [10]:
sql_query = 'SELECT * FROM customer;'
df_dim_customer = pd.read_sql(sql_query, mysql_driver)
df_dim_customer['name'] = df_dim_customer['first_name'] + ' ' + df_dim_customer['last_name']
df_dim_customer.drop(['dob', 'email', 'phone', 'first_name', 'last_name'], axis=1, inplace=True)
df_dim_customer = df_dim_customer.reindex(columns=['id', 'name', 'driver_license_number'])
df_dim_customer.rename(columns={'driver_license_number': 'driver_license'}, inplace=True)
df_dim_customer.head()

,id,name,driver_license
0,1,Kelby Matterdace,V435899293
1,2,Orion De Hooge,Z140530509
2,3,Sheena Macias,W045654959
3,4,Irving Packe,O232196823
4,5,Kass Humphris,G055017319


## Vehicle dim

In [11]:
sql_query = 'SELECT * FROM vehicle;'
df_vehicle = pd.read_sql(sql_query, mysql_driver)
df_vehicle.head()

,id,brand,model,model_year,mileage,color,vehicle_type_id,current_location_id
0,1,Nissan,Versa,2016,65956,white,1,1
1,2,Mitsubishi,Mirage,2017,55864,light blue,1,6
2,3,Chevrolet,Cruze,2017,45796,dark gray,2,5
3,4,Hyundai,Elantra,2018,35479,black,2,1
4,5,Volkswagen,Jetta,2019,2032,light gray,3,3


In [12]:
sql_query = 'SELECT * FROM vehicle_type;'
df_vehicle_type = pd.read_sql(sql_query, mysql_driver)
df_vehicle_type.head()

,id,name,rental_value
0,1,Economy,26.77
1,2,Intermediate,29.45
2,3,Standard,34.81
3,4,Economy SUV,37.48


In [13]:
df_dim_vehicle = df_vehicle_type.merge(df_vehicle, 
                                                left_on='id',
                                                right_on='vehicle_type_id',
                                                how='left',
                                                suffixes=('_type','_veh'))
df_dim_vehicle.drop(['id_veh', 'vehicle_type_id', 'current_location_id'], axis=1, inplace=True)
df_dim_vehicle.head()

,id_type,name,rental_value,brand,model,model_year,mileage,color
0,1,Economy,26.77,Nissan,Versa,2016,65956,white
1,1,Economy,26.77,Mitsubishi,Mirage,2017,55864,light blue
2,2,Intermediate,29.45,Chevrolet,Cruze,2017,45796,dark gray
3,2,Intermediate,29.45,Hyundai,Elantra,2018,35479,black
4,3,Standard,34.81,Volkswagen,Jetta,2019,2032,light gray


## Fuel dim

In [14]:
sql_query = 'SELECT * from fuel_option;'
df_dim_fuel = pd.read_sql(sql_query, mysql_driver)
df_dim_fuel.drop(['description'], axis=1, inplace=True)
df_dim_fuel.head()

,id,name
0,1,Pre-pay
1,2,Self-Service
2,3,Market


## Rental fact

In [15]:
sql_query = 'SELECT * FROM rental;'
df_fact_rental = pd.read_sql(sql_query, mysql_driver)

df_fact_rental['start_date_key'] = pd.to_datetime(df_fact_rental['start_date'], format='%Y-%m-%d').dt.strftime('%Y%m%d').astype(int)
df_fact_rental['end_date_key'] = pd.to_datetime(df_fact_rental['end_date'], format='%Y-%m-%d').dt.strftime('%Y%m%d').astype(int)

df_fact_rental.drop(['start_date', 'end_date'], axis=1, inplace=True)

df_fact_rental.head()

,id,customer_id,vehicle_type_id,fuel_option_id,pickup_location_id,drop_off_location_id,start_date_key,end_date_key
0,1,1,2,1,3,5,20180714,20180723
1,2,2,1,2,1,2,20180710,20180712
2,3,3,1,3,4,6,20180615,20180720
3,4,4,4,2,2,7,20180609,20180702
4,5,5,3,3,5,3,20180724,20180727


# Rental dataset

In [16]:
df_rental = df_fact_rental.merge(df_dim_customer, left_on='customer_id', right_on='id', how='left', suffixes=('_rental','_customer'))
df_rental = df_rental.merge(df_dim_vehicle, left_on='vehicle_type_id', right_on='id_type', how='left', suffixes=('_rental','_veh'))
df_rental = df_rental.merge(df_dim_fuel, left_on='fuel_option_id', right_on='id', how='left', suffixes=('_rental','_fuel'))
df_rental = df_rental.merge(df_dim_location, left_on='pickup_location_id', right_on='id_loc', how='left', suffixes=('_rental','_pick'))
df_rental = df_rental.merge(df_dim_location, left_on='drop_off_location_id', right_on='id_loc', how='left', suffixes=('_pick','_drop'))
df_rental = df_rental.merge(df_dim_date, left_on='start_date_key', right_on='date_key', how='left', suffixes=('_rental','_start'))
df_rental = df_rental.merge(df_dim_date, left_on='end_date_key', right_on='date_key', how='left', suffixes=('_start','_end'))

df_rental.head()

,id_rental,customer_id,vehicle_type_id,fuel_option_id,pickup_location_id,drop_off_location_id,start_date_key,end_date_key,id_customer,name_rental,...,day_name_start,quarter_start,year_start,date_key_end,full_date_end,day_of_week_end,day_num_in_month_end,day_name_end,quarter_end,year_end
0,1,1,2,1,3,5,20180714,20180723,1,Kelby Matterdace,...,Saturday,3,2018,20180723,2018-07-23,1,23,Monday,3,2018
1,1,1,2,1,3,5,20180714,20180723,1,Kelby Matterdace,...,Saturday,3,2018,20180723,2018-07-23,1,23,Monday,3,2018
2,2,2,1,2,1,2,20180710,20180712,2,Orion De Hooge,...,Tuesday,3,2018,20180712,2018-07-12,4,12,Thursday,3,2018
3,2,2,1,2,1,2,20180710,20180712,2,Orion De Hooge,...,Tuesday,3,2018,20180712,2018-07-12,4,12,Thursday,3,2018
4,3,3,1,3,4,6,20180615,20180720,3,Sheena Macias,...,Friday,2,2018,20180720,2018-07-20,5,20,Friday,3,2018


In [17]:
df_rental.drop(['customer_id', 'vehicle_type_id', 
                'fuel_option_id', 'pickup_location_id', 
                'drop_off_location_id', 'start_date_key', 
                'end_date_key', 'id_customer',
                'id_type', 'id', 'id_loc_pick',
                'id_loc_drop', 'date_key_start',
                'date_key_end'
            ], axis=1, inplace=True)

In [18]:
df_rental = df_rental.reindex(columns=['id_rental', 'name_rental', 'driver_license',
                                       'brand',	'model', 'model_year',	
                                       'mileage', 'color', 'name_veh', 'rental_value',	
                                       'name', 'street_address_pick', 'city_pick', 'state_pick',	
                                       'full_date_start', 'day_of_week_start', 'day_num_in_month_start',	
                                       'day_name_start', 'quarter_start', 'year_start',	
                                       'street_address_drop', 'city_drop', 'state_drop',	
                                       'full_date_end',	'day_of_week_end', 'day_num_in_month_end', 
                                       'day_name_end', 'quarter_end', 'year_end' 
                                    ])

In [20]:
df_rental.rename(columns={'name_rental': 'name_customer', 
                                'driver_license': 'license_customer',
                                'brand': 'brand_vehicle',	
                                'model': 'model_vehicle', 
                                'model_year': 'model_year_vehicle',	
                                'mileage': 'mileage_vehicle', 
                                'color': 'color_vehicle', 
                                'name_veh': 'type_vehicle', 	
                                'name': 'fuel_option', 
                                'street_address_pick': 'street_address_start', 
                                'city_pick': 'city_start', 
                                'state_pick': 'state_start',	
                                'full_date_start': 'date_start', 	
                                'street_address_drop': 'street_address_end', 
                                'city_drop': 'city_end', 
                                'state_drop': 'state_end' ,	
                                'full_date_end': 'date_end'
                            }, inplace=True)

# Upload S3 Rental dataset

In [21]:
df_rental.to_excel('data/df_rental.xlsx', index=False)

In [22]:
s3.Bucket(S3_BUCKET_NAME).upload_file('data/df_rental.xlsx', 'df_rental.xlsx')